In [2]:
import datetime
import pandas as pd
import numpy as np

from pyspark import SparkContext
from pyspark import SQLContext
from pyspark.sql.session import SparkSession


from pyspark.sql import DataFrameStatFunctions as statFunc

from pyspark.sql.functions import row_number
from pyspark.sql.window import Window
from pyspark.sql.functions import desc

from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.types import IntegerType,FloatType,DoubleType

## 1 Read Prediction

In [3]:
# This is the prediction from the model with data ranged from 10/01/2019 - 01/01/2020.
prediction = sqlContext.read.parquet('s3://ege-ds-workshops-corp/yixli/prediction/prediction')

### 1.1 Calculate normalized rate revenue

In [4]:
def calculateNormalizedRateRevenue(prediction):
    # sum probabilities by hotel
    prediction = prediction.withColumn('sum_prob',F.sum('prob').over(Window.partitionBy("message_id","hotel_id","check_in_date","check_out_date",'tuid')))

    # normalized rate-picking up probabilities
    prediction = prediction.withColumn('normalized_prob',F.col('prob')/F.col('sum_prob'))

    # normalized rate revenue
    prediction = prediction.withColumn('normalized_rate_revenue',F.col('normalized_prob')*F.col('src_supply_revenue_usd'))
    return prediction

### 1.2 Sum rate level revenue to hotel level revenue

In [8]:
def calculateNormalizedHotelRevenue(prediction):
    # get duration of stay
    prediction = prediction.\
                         withColumn('duration', F.datediff(F.col("check_out_date"),F.col( "check_in_date")).cast(IntegerType()))
    # get hotel revenue df by sum up rate revenue for each hotel
    revenue_prediction = prediction.withColumn('normalized_hotel_revenue',
                                       F.sum(F.col('normalized_rate_revenue')*F.col('duration')).\
                                       over(Window.partitionBy("message_id","hotel_id","check_in_date","check_out_date",'tuid'))).\
                            select("message_id","hotel_id","check_in_date","check_out_date","tuid",'normalized_hotel_revenue','hotel_index','score_1','bk_hotel_index','message_date').\
                            dropDuplicates()
    return revenue_prediction

## 2 Bands

In [10]:
def getBands(n_band,band_column,revenue_prediction):
    # exclude those hotels whose scores are NULL
    revenue_prediction = revenue_prediction.filter(F.col('score_1').isNotNull())
    # create band_df with upper and lower bounds of each band
    band_df = revenue_prediction.\
    groupby("message_id", "tuid").\
    agg(F.count("hotel_id").alias("n"),F.max("score_1").alias("ub"),F.min("score_1").alias("lb")).\
    filter(F.col("n")>=1).\
    filter(F.col("n")<=30).\
    withColumn("n_band", F.lit(n_band)).\
    withColumn("sz_band", F.expr("(ub - lb)/n_band")).\
    select("message_id", "tuid", "sz_band", "ub", "lb")
    # join hotel revenue df to get band for each hotel
    revenue_prediction2 = revenue_prediction.\
    join(band_df, ["message_id", "tuid"]).\
    withColumn(band_column, F.when(F.col("sz_band") == 0, 1).otherwise(F.expr("int((score_1-lb)/sz_band)")))
    return revenue_prediction2

## 3 New revenue with normalized probabilities

In [12]:
def getNewHotelIndex(revenue_prediction2,band_column,new_index_column):
    # add month column to df
    revenue_prediction2 = revenue_prediction2.withColumn('month',F.month('message_date'))
    # Sort by 'normalized_hotel_revenue' and obtain the new hotel index
    revenue_prediction2 = revenue_prediction2.\
    withColumn("band_index_normalized", row_number().over(Window.partitionBy("message_id", "tuid",band_column).\
                                                          orderBy(F.desc('normalized_hotel_revenue')))).\
    withColumn(new_index_column, row_number().over(Window.partitionBy("message_id", "tuid").\
                                                    orderBy(F.asc(band_column),F.asc("band_index_normalized"))))
    return revenue_prediction2

In [13]:
def getNewRevenue(revenue_prediction2,new_index_column):
    new_revenue = revenue_prediction2.filter(F.col('bk_hotel_index')==F.col(new_index_column)).\
                    select('normalized_hotel_revenue').\
                    agg(F.sum('normalized_hotel_revenue')).collect()[0][0]
    return new_revenue

In [27]:
prediction = calculateNormalizedRateRevenue(prediction)
revenue_prediction = calculateNormalizedHotelRevenue(prediction)
revenue_prediction2 = getBands(5,'band5',revenue_prediction)
revenue_prediction2 = getNewHotelIndex(revenue_prediction2,'band5','new_index_5')

In [28]:
# total supply revenue
revenue_prediction2.filter(F.col('bk_hotel_index')==F.col('new_index_5')).\
select('normalized_hotel_revenue').\
agg(F.sum('normalized_hotel_revenue')).show()

+-----------------------------+
|sum(normalized_hotel_revenue)|
+-----------------------------+
|         2.8679587695917096E7|
+-----------------------------+

In [26]:
# supply revenue by month
revenue_prediction2.filter(F.col('bk_hotel_index')==F.col('new_index_5')).\
select('normalized_hotel_revenue','month').\
groupBy('month').agg(F.sum('normalized_hotel_revenue'),F.count('month')).show()

+-----+-----------------------------+------------+
|month|sum(normalized_hotel_revenue)|count(month)|
+-----+-----------------------------+------------+
|   12|           7795188.4543378055|      130326|
|    1|            10532.18705316058|          99|
|   10|         1.1140306050966235E7|      183587|
|   11|            9733561.003559899|      167541|
+-----+-----------------------------+------------+

In [39]:
# save hotel revenue df
dir = 's3://ege-ds-workshops-corp/yixli/prediction/'
revenue_prediction2.repartition(1).write.mode('overwrite').parquet(dir+'revenue_estimation3')

## 5 Banding method exploration

In [9]:
prediction = calculateNormalizedRateRevenue(prediction)
revenue_prediction = calculateNormalizedHotelRevenue(prediction)

In [22]:
def estimateNewrevenue(revenue_prediction,band_num):
    band_column = 'band'+str(band_num)
    new_index_column = 'band'+str(band_num)+'_new_index'
    revenue_prediction2 = getBands(band_num,band_column,revenue_prediction)
    revenue_prediction2 = getNewHotelIndex(revenue_prediction2,band_column,new_index_column)
    booking_num = revenue_prediction2.filter(F.col('bk_hotel_index')==F.col(new_index_column)).count()
    new_revenue = getNewRevenue(revenue_prediction2,new_index_column)
    return booking_num, new_revenue 

In [23]:
band_num_list = [1,2,3,4,5,6,7,8,9,10]
new_revenue_list = []
booking_num_list = []
for band_num in band_num_list:
    booking_num, new_revenue  = estimateNewrevenue(revenue_prediction,band_num)
    new_revenue_list.append(new_revenue)
    booking_num_list.append(booking_num)

In [24]:
booking_num_list

[481527, 481541, 481564, 481556, 481553, 481541, 481528, 481555, 481564, 481569]

In [25]:
new_revenue_list

[32361865.630446505, 30482008.659255844, 29554993.28257789, 29054868.17835278, 28679587.695917077, 28378901.6701945, 28125813.61617068, 27961289.39853387, 27773488.968403537, 27667802.009333476]